In [45]:
import pandas as pd

from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)
import string
punct=string.punctuation
from spacy.lang.en import English

import gensim
import pyLDAvis
#import pyLDAvis.gensim
import pyLDAvis.gensim_models
from gensim.utils import simple_preprocess

import warnings
warnings.filterwarnings("ignore")

In [3]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [4]:
data = pd.read_csv('ted_talks_en.csv')
data.head(2)

,talk_id,title,speaker_1,all_speakers,occupations,about_speakers,views,recorded_date,published_date,event,native_lang,available_lang,comments,duration,topics,related_talks,url,description,transcript
0,1,Averting the climate crisis,Al Gore,{0: 'Al Gore'},{0: ['climate advocate']},{0: 'Nobel Laureate Al Gore focused the world’...,3523392,2006-02-25,2006-06-27,TED2006,en,"['ar', 'bg', 'cs', 'de', 'el', 'en', 'es', 'fa...",272.0,977,"['alternative energy', 'cars', 'climate change...","{243: 'New thinking on the climate crisis', 54...",https://www.ted.com/talks/al_gore_averting_the...,With the same humor and humanity he exuded in ...,"Thank you so much, Chris. And it's truly a gre..."
1,92,The best stats you've ever seen,Hans Rosling,{0: 'Hans Rosling'},{0: ['global health expert; data visionary']},"{0: 'In Hans Rosling’s hands, data sings. Glob...",14501685,2006-02-22,2006-06-27,TED2006,en,"['ar', 'az', 'bg', 'bn', 'bs', 'cs', 'da', 'de...",628.0,1190,"['Africa', 'Asia', 'Google', 'demo', 'economic...","{2056: ""Own your body's data"", 2296: 'A visual...",https://www.ted.com/talks/hans_rosling_the_bes...,You've never seen data presented like this. Wi...,"About 10 years ago, I took on the task to teac..."


In [5]:
data4 = data['transcript'].astype(str)

In [6]:
data5 = data4[0:400]

In [7]:
# transcripts = pd.DataFrame(data['transcript'])
# transcripts

In [8]:
# transcripts = transcripts.astype(str)

In [9]:
# data3 = str(transcripts)

In [10]:
# data3

In [11]:
# data['transcript']= data['transcript'].astype(str)

In [12]:
# data1 = data['transcript']

In [13]:
# #data = load_data("data/ushmm_dn.json")["texts"]

# print (data1[1][0:90])

In [14]:
# trans_list = transcripts['transcript'].tolist()
# trans_list2 = trans_list[0:10]

In [15]:
def tokenizerX(x):
    doc = nlp(x)
    
    tokens = [token.lemma_.lower().strip() if token.lemma_ != "-PRON-" else token.lower_ for token in doc]     
    final_tokens = [token for token in tokens if token not in stopwords and token not in punct]
    return final_tokens

In [16]:
import en_core_web_sm
from tqdm import tqdm

## How to use spaCy on text/strings stored in a data frame:

In [18]:
# Any text cleaning function can be applied to non string type data using this
tqdm.pandas()
data["processed_text"] = data["transcript"].progress_apply(tokenizerX)

100%|███████████████████████████████████████| 4005/4005 [23:42<00:00,  2.81it/s]


In [22]:
texts = data['processed_text'].values
#max_features = 2**12

## Alternate way to apply spaCy function to text stored in a dataframe:

In [23]:
# how to apply the tokenize function on strings/ text stored in a data frame 
# use .apply() method 
# you can take the column that has all the text and apply the tokenize function on it. 
'''
data['tokens'] = data['comment_text'].apply(spacy_tokenizer)
'''


"\ndata['tokens'] = data['comment_text'].apply(spacy_tokenizer)\n"

In [46]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True) #deaccent removed any accent from polish/ french words
        final.append(new)
    return (final)

data_words = gen_words(texts)

print (data_words[0][0:20])

TypeError: decoding to str: need a bytes-like object, list found

In [47]:
data_words = texts

In [38]:
# bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
# # the first argument here is going to see if there is an overlap between bigrams and other words that have existed 
# # and then figure when three words have been used collectively to mean something 
# trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

# bigram = gensim.models.phrases.Phraser(bigram_phrases)
# trigram = gensim.models.phrases.Phraser(trigram_phrases)

# def make_bigrams(texts):
#     return([bigram[doc] for doc in texts])

# def make_trigrams(texts):
#     return ([trigram[bigram[doc]] for doc in texts])

# data_bigrams = make_bigrams(data_words)
# data_bigrams_trigrams = make_trigrams(data_bigrams)

# print (data_bigrams_trigrams[0][0:20])

In [ ]:
# from gensim.models import TfidfModel

# id2word = corpora.Dictionary(data_bigrams_trigrams) 

# texts = data_bigrams_trigrams

# corpus = [id2word.doc2bow(text) for text in texts]
# # print (corpus[0][0:20])

# tfidf = TfidfModel(corpus, id2word=id2word)

# low_value = 0.03
# words  = []
# words_missing_in_tfidf = []
# for i in range(0, len(corpus)):
#     bow = corpus[i]
#     low_value_words = [] #reinitialize to be safe. You can skip this.
#     tfidf_ids = [id for id, value in tfidf[bow]]
#     bow_ids = [id for id, value in bow]
#     low_value_words = [id for id, value in tfidf[bow] if value < low_value]
#     drops = low_value_words+words_missing_in_tfidf
#     for item in drops:
#         words.append(id2word[item])
#     words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

#     new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
#     corpus[i] = new_bow

In [48]:
# this code cell is used when we do topic modleing without bigrams, trigrams and tf-idf word removal 


import gensim.corpora as corpora

id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)



[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 4)]
2.0


In [49]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")


### Visualization

In [50]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

/Users/punamichowdary/Applications/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/punamichowdary/Applications/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/punamichowdary/Applications/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/punamichowdary/Applications/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Use

/Users/punamichowdary/Applications/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/punamichowdary/Applications/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/punamichowdary/Applications/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/punamichowdary/Applications/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/punamichowdary/Applications/anaconda3/lib/python3.9/site-

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14     0.017411  0.354863       1        1  19.602249
11    -0.003017  0.202563       2        1  17.127680
9     -0.219809  0.257409       3        1  12.188820
4     -0.179516  0.141934       4        1   7.870963
3      0.162643  0.154445       5        1   6.543882
12    -0.054663  0.058652       6        1   6.367643
15     0.276415  0.053119       7        1   5.163077
7     -0.155309  0.344569       8        1   4.200628
5      0.325396  0.195584       9        1   3.832414
6      0.077215 -0.107367      10        1   3.585810
18     0.235882 -0.175887      11        1   2.899507
17     0.244152  0.345946      12        1   2.471774
13     0.272867 -0.287091      13        1   1.996749
2     -0.265008 -0.071807      14        1   1.903550
10     0.412399 -0.081050      15        1   1.776421
8     -0.454827  0.091378      16        1   1.427365
19    -0.214142 -0.413253      17        1   0.625870
1     -0.362644 -0.273411      18        1   0.413918
16    -0.057398 -0.395705      19        1   0.001225
0     -0.058047 -0.394891      20        1   0.000455, topic_info=            Term          Freq         Total Category  logprob  loglift
327       people  30964.000000  30964.000000  Default  30.0000  30.0000
479          use  17055.000000  17055.000000  Default  29.0000  29.0000
1326        word   3857.000000   3857.000000  Default  28.0000  28.0000
457        think  24799.000000  24799.000000  Default  27.0000  27.0000
242     laughter  16277.000000  16277.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
25       anybody      0.000231    590.657050  Topic20 -10.9565  -2.4518
26           app      0.000231    548.199925  Topic20 -10.9565  -2.3772
27      applause      0.000231   8193.729668  Topic20 -10.9565  -5.0817
28    appreciate      0.000231    247.169375  Topic20 -10.9565  -1.5806
29      approach      0.000231   1398.800306  Topic20 -10.9565  -3.3139

[1424 rows x 6 columns], token_table=       Topic      Freq Term
term                       
882        1  0.704082  ...
882        2  0.118243  ...
882        3  0.126842  ...
882        4  0.000717  ...
882       17  0.049805  ...
...      ...       ...  ...
512       13  0.001359    —
512       14  0.000107    —
512       15  0.000393    —
512       16  0.004934    —
10450     17  0.996314    ♫

[3806 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 12, 10, 5, 4, 13, 16, 8, 6, 7, 19, 18, 14, 3, 11, 9, 20, 2, 17, 1])

## Other Text Cleaning Methods (from this topic modeling github)

In [ ]:
# from this topic modeling github 
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

In [ ]:
lemmatized_texts = lemmatization(data5)


In [ ]:
print (lemmatized_texts[0][0:90])

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

### gensim.utils.simple_preprocess( )

- Converts a document into a list of tokens.
- This lowercases, tokenizes, de-accents (optional). 
- the output are final tokens = unicode strings, that won’t be processed any further

In [ ]:
# # LDA takes in a list of unicode strings 
# summaries = []
# for line in trans_list2:
#    temp = line.split("\t")
#    summaries.append(tokenizerX(temp[0]))

In [ ]:
# we need to do some more preprocesssing to make the text ready for topic modeling 
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True) #deaccent removed any accent from polish/ french words
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

In [ ]:
import gensim.corpora as corpora

id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")


## Visualizing the data

In [ ]:
import pyLDAvis
#import pyLDAvis.gensim
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings("ignore")

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

## Another way to clean data

In [ ]:
import string

punctuations = string.punctuation
stopwords = list(STOP_WORDS)
stopwords[:10]

In [ ]:
# Parser
parser = en_core_web_sm.load(disable=["tagger", "ner"])
parser.max_length = 7000000

def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens